In [ ]:
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
url = "http://grafins.com/ml/compressed_data.zip"
with urlopen(url) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('.')

In [ ]:
import numpy as np
import pandas as pd 
import os
import cv2
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import *
import matplotlib.pyplot as plt
import math
import random

from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
data_dir = './compressed_data/train/'
img_size = 75
def get_data():
  data = []
  x_data = []
  y_data = []
  i=0;
  for label in labels:
    path = os.path.join(data_dir,label)
    print("Extracting",label)
    for img in os.listdir(path):
      try:
        image = cv2.imread(os.path.join(path,img))
        image = cv2.resize(image,(img_size,img_size))
        data.append([image,i])
      except Exception as e:
        print(e)
    i+=1
  random.shuffle(data)
  for d in data:
    x_data.append(d[0])
    y_data.append(d[1])
  print(len(x_data),len(y_data))
  return np.array(x_data),np.array(y_data),len(x_data)

  



In [ ]:
x, y, size = get_data()

Extracting A
Extracting B
Extracting C
Extracting D
Extracting E
Extracting F
Extracting G
Extracting H
Extracting I
Extracting J
Extracting K
Extracting L
Extracting M
Extracting N
Extracting O
Extracting P
Extracting Q
Extracting R
Extracting S
Extracting T
Extracting U
Extracting V
Extracting W
Extracting X
Extracting Y
Extracting Z
78000 78000


In [ ]:
split = math.ceil(size*0.9)
x_train = x[:split]
y_train = y[:split]
x_test = x[split:]
y_test = y[split:]

In [ ]:
model = InceptionV3(input_shape = (img_size,img_size,3),include_top = False,weights='imagenet')

for layer in model.layers[:-55]:
  layer.trainable = False;

In [ ]:
len(model.layers[:-55])

260

In [ ]:

last_layer = keras.layers.Flatten()(model.output)
last_layer = keras.layers.Dense(4096,activation='relu')(last_layer)
last_layer = keras.layers.Dense(512,activation='relu')(last_layer)
last_layer = keras.layers.Dense(26,activation='softmax')(last_layer)
model = keras.Model(model.input,last_layer)

In [ ]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 75, 75, 3)]  0           []                               
                                                                                                  
 conv2d_846 (Conv2D)            (None, 37, 37, 32)   864         ['input_10[0][0]']               
                                                                                                  
 batch_normalization_846 (Batch  (None, 37, 37, 32)  96          ['conv2d_846[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_846 (Activation)    (None, 37, 37, 32)   0           ['batch_normalization_846[0

In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_split = 0.2)

In [44]:
model.evaluate(x_test,y_test)

244/244 [==============================] - 12s 47ms/step - loss: 0.1420 - accuracy: 0.9536


[0.14200496673583984, 0.9535897374153137]

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

In [ ]:

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper right')